In [2]:
%pylab inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


# COMP 598 - Lecture 5: Generative Models for Linear Classification


Last time we saw  *Logistic Regression* which was a discriminative learning technique that tried to estimate $P(Y~|~X)$. This time we will learn about **generative learning** where we aim to model $P(X~|~Y)$ and $P(y)$ and use the two together to estimate $P(Y~|~X)$ via Baye's Rule.

## Linear Discriminant Analysis (LDA)

Assume we have two classes $Y\in(0,1)$. We can find the probabilities of finding each class:
$$P(Y=0) = \frac{N(Y=0)}{N(Y=1) + N(Y=0)}\\
P(Y=1) = \frac{N(Y=1)}{N(Y=1) + N(Y=0)}$$

In LDA we assume that our data, $X$, was generated by a Multivariate Gaussian:

$$P(X~|~Y) = \frac{ \exp{ \Big( -\frac{1}{2} (x - \mu )^T \Sigma^{-1} (x-\mu) \Big) }} { (2\pi)^{\frac{1}{2}} |\Sigma|^{\frac{1}{2}} }$$

Note here that the key assumption about LDA is that they have the same covariance matrix $\Sigma$

## Naive Bayes (to be written using Mitchells notes)
This is another generative learning model. Here our goal is to estimate $P(Y|X)$, i.e. how likely are we to observe a certain class, $Y$, given we certain features $X$.

Recall Bayes Rule: $ P(Y~|~X) = \frac{P(X~|~Y) P(Y)}{P(X)}$. 

We have three components, 
$$P(Y) := \text{The probability of observing class $Y$}\\
P(X) := \text{Probability of observing the features $X$}\\
P(X~|~Y) := \text{The probability of observing features $X$ in a certain class $Y$}
$$

**The Naive Bayes Assumption:** Here we assume that the features of $X$, $X_j$ are conditionally independent given $Y$. This means:

$$P(X~|~Y) = P(X_1, X_2, X_3, \ldots, X_m | Y) = \prod_{j=1}^n P(X_j~|~Y) $$


Thus now: 

$$P(Y~|~X) = \frac{P(X~|~Y) P(Y)}{P(X)} = \frac{P(Y) \prod_{j=1}^n P(X_j~|~Y) }{P(X)}$$


Our goal now is to estimate the individual $P(X_j~|~Y)$ and $P(Y)$ from the data. We define the estimators:
$$
\theta_1 := P(Y=1) \\
\theta_{j,1} := P(X_j=1~|~Y=1) \\
\theta_{j,0} := P(X_j=0~|~Y=0)
$$

Above which can be obtained by counting. We now need an evaluation criteria. For example, here we can find the parameters that maximize the log-likelihood function:

Likelihood: $P(Y~|~X) \propto \prod_{i=1}^n \Big( P(y_i) \prod_{j=1}^m P(x_{i,j}~|~y_i) \Big)$
Reasoning:
- Bayes theorem, $P(X)$ in the denominator is disregarded.
- The samples $i$ are independent, thus we take a product over the $n$ $y_i$'s
- Input features are independent, thus we can take a product over $m$ for each feature $x_{i,j}$.

Log-likelihood: $\log L(\theta_1, \theta_{j,1}, \theta_{j,0}) = \sum_{i=1}^n \Big( \log P(y_i) + \sum_{j=1}^m \log P(x_{i,j}~|~y_i) \Big)$

- To estimate $\theta_1$, we need to take the derivative: of $\log L$: 
$$\frac{\delta L}{\delta \theta_1} = \sum_{j=1}^n \Big(\frac{y_i}{\theta_1} - \frac{(1-y_i)}{(1-\theta_1)} \Big) = 0 $$

Solving for $\theta_1$, we obtain: $\theta_1 = \frac{1}{n} \sum_{i=1}^n y_i$ i.e the proportion of examples where $y=1$. By using simillar logic we can find $\theta_{j,1}$ and $\theta_{j,0}$ which are the proportion of examples where $x_j = 1$ and $y = \{0,1\}$ respectively.

Now how do we make decisions? Consider the log-odds ratio:

$$ 
\log \frac{P(X~|~Y=1) P(Y=1)}{P(X~|~Y=0) P(Y=0)} = \log \frac{ P(Y=1) }{ P(Y=0) } + \log \frac{ \prod_{j=1}^m P(x_j~|~Y=1) }{ \prod_{j=1}^m P(x_j~|~y=0) }\\
= log \frac{ P(Y=1) }{ P(Y=0) } + \sum_{j=1}^m \log \frac{  P(x_j~|~Y=1) }{ P(x_j~|~y=0) }
$$

Now notice that: **NEED TO DO WORK HERE ON DERIVATION**
$$\sum_{j=1}^m \log \frac{  P(x_j~|~Y=1) }{ P(x_j~|~y=0)} = $$

By letting $w_{j,0} = \log \frac{ P( x_j = 0 | y = 1) } {P(x_j = 0 | y = 0)}$ and $w_{j,1} = \log \frac{ P(x_j=1 | y=1) }{ P(x_j=1 | y=0) }$ we obtain:



$$ = log \frac{ P(Y=1) }{ P(Y=0) } + \sum_{j=1}^m \Big( w_{j,0}(1-x_j) + w_{j,1}x_j  \Big)\\
= \underbrace{log \frac{ P(Y=1) }{ P(Y=0) } + \sum_{j=1}^m w_{j,0} }_{w_0} + \underbrace{ \sum_{j=1}^m \Big( w_{j,1} - w_{j,0}\Big)x_j }_{X^TW} 
$$

$w_0$ and $W$ are the weights of a linear boundary give our features $x_1, \ldots, x_j$.

## Simple Example: Learning a Boolean Function:

Assume we have $y^* = x_1\oplus x_2$, the logical operator for XOR, exclusive OR.

In [63]:
X = np.array([np.random.randint(2, size=50), np.random.randint(2, size=50)])
Y = 1*np.logical_xor( X[0], X[1] )
X = X.T


# unique labels in our set
# this will hold the references for the label indicies.
unique_labels = np.unique(Y) 

data_array = np.array(X) # the data in array format


n = data_array.shape[0] # number of observations we have
m = data_array.shape[1] # number of features
k = len(unique_labels) # number of classes
t = 2 #assuming x_i are boolean variables

labels_array = np.array(Y)

In [65]:
gamma = 1 #smoothening parameter

theta = np.zeros((m,t,k))
pie = np.zeros(k)

In [66]:
from collections import Counter

In [117]:
unique_feature_labels = [None]*m # array to hold the unique values of each feature

for label_index, label in enumerate(unique_labels):
    print 'y_k=',label
    matching_indicies = labels_array == label
    pie[label_index] = (np.sum(matching_indicies)+gamma)/(float(n)+gamma*k)
    print 'pie[k]=',pie[label_index]
    # obtain all the data that matches our labels.
    data_for_label = data_array[matching_indicies]
    
    # transpose the matrix because we want to now calculate the counts of the features.
    feature_observations = data_for_label.T
    
    for feature_index, feature_observation in enumerate(feature_observations):
        
        unique_feature_labels[feature_index] = list(sorted(np.unique(feature_observation)))
        
        for count in Counter(feature_observation).items():
            t_instance, N_condition = count[0], count[1] # value, frequency 

            theta[feature_index, t_instance, label_index] = (N_condition + gamma)/float(np.sum(matching_indicies) + gamma * t)
            print 'theta(j=',feature_index,', t=',t_instance,'k=',label_index,') = ',theta[feature_index, t_instance, label_index]
            


y_k= 0
pie[k]= 0.384615384615
theta(j= 0 , t= 0 k= 0 ) =  0.666666666667
theta(j= 0 , t= 1 k= 0 ) =  0.333333333333
theta(j= 1 , t= 0 k= 0 ) =  0.666666666667
theta(j= 1 , t= 1 k= 0 ) =  0.333333333333
y_k= 1
pie[k]= 0.615384615385
theta(j= 0 , t= 0 k= 1 ) =  0.545454545455
theta(j= 0 , t= 1 k= 1 ) =  0.454545454545
theta(j= 1 , t= 0 k= 1 ) =  0.454545454545
theta(j= 1 , t= 1 k= 1 ) =  0.545454545455


In [68]:
theta

array([[[ 0.66666667,  0.54545455],
        [ 0.33333333,  0.45454545]],

       [[ 0.66666667,  0.45454545],
        [ 0.33333333,  0.54545455]]])

In [116]:
idx = 7
classify_this = X[idx]
print 'to_classify:',classify_this
results = []
for label in unique_labels:
    p = pie[label]
    print 'P(y=',label,')=',pie[label]
    for j, val in zip(range(m), classify_this):
        print 'P(X_',j,'=',val,' | y=',label,')=', theta[j,val,label]
        p = p * theta[j,val,label]
    print '\n'
    results.append((label, p))

print 'Ps: ',results

print 'Prediction:', max(results,key=lambda l: l[1])
print 'Actual:',Y[idx]

to_classify: [0 0]
P(y= 0 )= 0.384615384615
P(X_ 0 = 0  | y= 0 )= 0.666666666667
P(X_ 1 = 0  | y= 0 )= 0.666666666667


P(y= 1 )= 0.615384615385
P(X_ 0 = 0  | y= 1 )= 0.545454545455
P(X_ 1 = 0  | y= 1 )= 0.454545454545


Ps:  [(0, 0.17094017094017092), (1, 0.15257469802924348)]
Prediction: (0, 0.17094017094017092)
Actual: 0


In [86]:
from sklearn.naive_bayes import BernoulliNB
nb = BernoulliNB()
nb_trained = nb.fit(X, Y.T)
nb_trained.predict_proba(X[0])

array([[ 0.31402814,  0.68597186]])

In [96]:
Y[0], X[0], nb_trained.predict(X[0])

(1, array([0, 1]), array([1]))

In [97]:
Y[1], X[1], nb_trained.predict(X[1])

(0, array([0, 0]), array([0]))

In [98]:
Y[2], X[2], nb_trained.predict(X[2])

(1, array([1, 0]), array([1]))

In [106]:
Y[8], X[8], nb_trained.predict(X[8])

(0, array([1, 1]), array([1]))

A good check is if our probability estimates are good:

In [127]:
np.exp(nb_trained.class_log_prior_)

array([ 0.38,  0.62])

In [128]:
pie

array([ 0.38461538,  0.61538462])

Looks GOOD! How about our feature estimates?

In [129]:
np.exp(nb_trained.feature_log_prob_)

array([[ 0.33333333,  0.33333333],
       [ 0.45454545,  0.54545455]])

In [130]:
theta

array([[[ 0.66666667,  0.54545455],
        [ 0.33333333,  0.45454545]],

       [[ 0.66666667,  0.45454545],
        [ 0.33333333,  0.54545455]]])

Looks pretty decent if you match up which ones are the positive labels

## Example: Sex Determination


In [10]:
df = pd.DataFrame([['Drew', 'No', 'Blue', 'Short', 'Male',],
['Claudia', 'Yes', 'Brown', 'Long', 'Female',],
['Drew', 'No', 'Blue', 'Long', 'Female',],
['Drew', 'No', 'Blue', 'Long', 'Female',],
['Alberto', 'Yes', 'Brown', 'Short', 'Male',],
['Karin', 'No', 'Blue', 'Long', 'Female',],
['Nina', 'Yes', 'Brown', 'Short', 'Female',],
['Sergio', 'Yes', 'Blue', 'Long', 'Male',]])
df.columns = ['Name', 'isOver170', 'EyeColor', 'HairLength', 'Sex']

In [11]:
df

,Name,isOver170,EyeColor,HairLength,Sex
0,Drew,No,Blue,Short,Male
1,Claudia,Yes,Brown,Long,Female
2,Drew,No,Blue,Long,Female
3,Drew,No,Blue,Long,Female
4,Alberto,Yes,Brown,Short,Male
5,Karin,No,Blue,Long,Female
6,Nina,Yes,Brown,Short,Female
7,Sergio,Yes,Blue,Long,Male
